# Echochambers Episode 1: The Optimization Menace
Echochambers are the usually most vilifying and useless form of communities. Little critical thinking is invested into the ideas circulated beyond the justification of already held beliefs, and any conflicting ideas are usually ridiculed or just ignored at worst. Platforms like Twitter and Reddit provide safe haven for a many echochambers on the internet as they make it incredibly easy to become intrenched in certain communities (like subreddits or twitter sub-cultures).

Here I attempt to model some of the behaviours of echochambers: simply at first, then building up complexity. I will see if I can find some non-intuitive and interesting conclusions while doing so.

A lot of the initial modelling will come from Frank Witte's ECON0055 Economics of Science lectures

In [ ]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import matplotlib
from IPython import display

In [ ]:
learning_rate = 1e-4
num_agents = 10

Initially, we will model agents in a relatively simple way. Each of the ten agents will have a fixed evidence variable, which will be relatively small random value (0 mean and 0.1 SD). Then, they will have another attribute called their belief, which is another small random value (0 mean and 0.1 SD) that represents which way on a particular issue they believe in. In this simulation, the issue will just have two opposing viewpoints, and will be a continuous number from -inf to +inf representing which side and how strong they believe in the viewpoint.

In [ ]:
beliefs = torch.divide(torch.randn(1, num_agents), 10).requires_grad_()
evidence = torch.divide(torch.randn(1, num_agents), 10)

In [ ]:
def plot(beliefs, evidence):
    fig, axs = plt.subplots(2)
    axs[0].imshow(beliefs.detach(), cmap="bwr", norm=matplotlib.colors.Normalize(-1, 1, True))
    axs[0].set_title("Agent beliefs", fontsize="x-large")
    axs[0].set_xlabel("Agent number")
    axs[0].set_xticks(range(beliefs.shape[1]))
    axs[0].tick_params(axis="y", which="both", left=False, right=False, labelleft=False)
    axs[1].imshow(evidence.detach(), cmap="bwr", norm=matplotlib.colors.Normalize(-1, 1, True))
    axs[1].set_title("Agent evidence", fontsize="x-large")
    axs[1].set_xlabel("Agent number")
    axs[1].set_xticks(range(evidence.shape[1]))
    axs[1].tick_params(axis="y", which="both", left=False, right=False, labelleft=False)
    plt.show()

print("Initial beliefs and evidence:")
plot(beliefs, evidence)

Finally, we get them to maximize their utility. Their utility will be defined simply as the product of their beliefs and the evidence for that belief. It makes sense that for them to maximize their utility, they will have to change their beliefs to fit the little evidence that they have by making it the same sign. After each "timeslot", they will look at which way they should change their beliefs to increase their utility and make a small step towards it.

In [ ]:
for t in range(100000):

    if t % 20000 == 19999:
        display.clear_output(wait=True)
        print(t, "steps")
        plot(beliefs, evidence)
        print("Average agent utility:", utility.mean().item())

    utility = beliefs * evidence
    
    # Find gradients for the beliefs to increase utility
    utility.backward(torch.ones_like(utility))

    # Update weights using gradient descent
    with torch.no_grad():
        beliefs += learning_rate * beliefs.grad
        beliefs.grad.zero_()


As we can see, they extrapolate their beliefs no matter how little evidence that they have in order to increase their utility, becoming more and more "radicalized"